<a href="https://colab.research.google.com/github/rajdeepbasu/Transformers/blob/main/Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files, drive
drive.mount('/content/gdrive')
# files.upload()

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
!pip install transformers
!pip install silence_tensorflow

In [3]:
%matplotlib inline
#Importing Transformers
import transformers
from transformers import AutoConfig, AutoModelForSequenceClassification, AutoTokenizer

#Importing Pandas
import pandas as pd

#Importing Numpy
import numpy as np

from silence_tensorflow import silence_tensorflow
silence_tensorflow()
import torch
import json
import nltk
import re
import string
import matplotlib.pyplot as plt
import operator

In [4]:
config = AutoConfig.from_pretrained("ProsusAI/finbert")
tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")

In [5]:
SBI_news = pd.read_csv('/content/gdrive/MyDrive/StockMarket/Abhishek/News Articles/SBI_GoogleNews_v2.csv')
SBI_news = SBI_news.loc[:, ~SBI_news.columns.str.contains('^Unnamed')]

In [6]:
SBI_news.head()

,title,media,date,datetime,desc,link,img
0,"India sets out Yes Bank rescue, State Bank of ...",Reuters,"Mar 6, 2020",2020-03-06 00:00:00.000000,"India sets out Yes Bank rescue, State Bank of ...",https://www.reuters.com/article/us-yesbank-ind...,"data:image/gif;base64,R0lGODlhAQABAIAAAP//////..."
1,"Yes Bank | In RBI bailout, SBI to pick up 49% ...",The Hindu,"Mar 6, 2020",2020-03-06 00:00:00.000000,"The SBI, which had earlier said its Board was ...",https://www.thehindu.com/business/Industry/yes...,"data:image/gif;base64,R0lGODlhAQABAIAAAP//////..."
2,State Bank of India to buy 49% stake in Yes Bank,ThePrint,"Mar 6, 2020",2020-03-06 00:00:00.000000,SBI has already obtained an 'in-principle' app...,https://theprint.in/economy/state-bank-of-indi...,"data:image/gif;base64,R0lGODlhAQABAIAAAP//////..."
3,Fitch Affirms State Bank of India's IDR at 'BB...,NaN,"Apr 30, 2020",2020-04-30 00:00:00.000000,Fitch Ratings - Singapore - 30 Apr 2020: Fitch...,https://www.fitchratings.com/research/banks/fi...,"data:image/gif;base64,R0lGODlhAQABAIAAAP//////..."
4,SBI cautions against fake UPI ID for collectin...,Livemint,"Mar 31, 2020",2020-03-31 00:00:00.000000,Country's largest lender State Bank of India (...,https://www.livemint.com/industry/banking/sbi-...,"data:image/gif;base64,R0lGODlhAQABAIAAAP//////..."


In [7]:
SBI_news_mod = SBI_news.copy(deep=True)

In [8]:
classes=['positive','negative','neutral']

In [9]:
SBI_news_mod['title_encoded'] = SBI_news_mod['title'].apply(lambda x: tokenizer.encode_plus(x,
                                                                                            truncation=True,
                                                                                            padding=True,
                                                                                            return_tensors="pt"))

In [10]:
SBI_news_mod['sentiment'] = SBI_news_mod['title_encoded'].apply(lambda x: dict(zip(classes, torch.softmax(model(**x)[0], dim=1).tolist()[0])))

In [11]:
SBI_news_mod.head()

,title,media,date,datetime,desc,link,img,title_encoded,sentiment
0,"India sets out Yes Bank rescue, State Bank of ...",Reuters,"Mar 6, 2020",2020-03-06 00:00:00.000000,"India sets out Yes Bank rescue, State Bank of ...",https://www.reuters.com/article/us-yesbank-ind...,"data:image/gif;base64,R0lGODlhAQABAIAAAP//////...","[input_ids, token_type_ids, attention_mask]","{'positive': 0.05125408247113228, 'negative': ..."
1,"Yes Bank | In RBI bailout, SBI to pick up 49% ...",The Hindu,"Mar 6, 2020",2020-03-06 00:00:00.000000,"The SBI, which had earlier said its Board was ...",https://www.thehindu.com/business/Industry/yes...,"data:image/gif;base64,R0lGODlhAQABAIAAAP//////...","[input_ids, token_type_ids, attention_mask]","{'positive': 0.5236324071884155, 'negative': 0..."
2,State Bank of India to buy 49% stake in Yes Bank,ThePrint,"Mar 6, 2020",2020-03-06 00:00:00.000000,SBI has already obtained an 'in-principle' app...,https://theprint.in/economy/state-bank-of-indi...,"data:image/gif;base64,R0lGODlhAQABAIAAAP//////...","[input_ids, token_type_ids, attention_mask]","{'positive': 0.07688067108392715, 'negative': ..."
3,Fitch Affirms State Bank of India's IDR at 'BB...,NaN,"Apr 30, 2020",2020-04-30 00:00:00.000000,Fitch Ratings - Singapore - 30 Apr 2020: Fitch...,https://www.fitchratings.com/research/banks/fi...,"data:image/gif;base64,R0lGODlhAQABAIAAAP//////...","[input_ids, token_type_ids, attention_mask]","{'positive': 0.48873522877693176, 'negative': ..."
4,SBI cautions against fake UPI ID for collectin...,Livemint,"Mar 31, 2020",2020-03-31 00:00:00.000000,Country's largest lender State Bank of India (...,https://www.livemint.com/industry/banking/sbi-...,"data:image/gif;base64,R0lGODlhAQABAIAAAP//////...","[input_ids, token_type_ids, attention_mask]","{'positive': 0.03070661798119545, 'negative': ..."


In [12]:
SBI_news_mod['sentiment_news'] = SBI_news_mod['sentiment'].apply(lambda x: max(x.items(), key=operator.itemgetter(1))[0])

In [13]:
SBI_news_mod.head()

,title,media,date,datetime,desc,link,img,title_encoded,sentiment,sentiment_news
0,"India sets out Yes Bank rescue, State Bank of ...",Reuters,"Mar 6, 2020",2020-03-06 00:00:00.000000,"India sets out Yes Bank rescue, State Bank of ...",https://www.reuters.com/article/us-yesbank-ind...,"data:image/gif;base64,R0lGODlhAQABAIAAAP//////...","[input_ids, token_type_ids, attention_mask]","{'positive': 0.05125408247113228, 'negative': ...",neutral
1,"Yes Bank | In RBI bailout, SBI to pick up 49% ...",The Hindu,"Mar 6, 2020",2020-03-06 00:00:00.000000,"The SBI, which had earlier said its Board was ...",https://www.thehindu.com/business/Industry/yes...,"data:image/gif;base64,R0lGODlhAQABAIAAAP//////...","[input_ids, token_type_ids, attention_mask]","{'positive': 0.5236324071884155, 'negative': 0...",positive
2,State Bank of India to buy 49% stake in Yes Bank,ThePrint,"Mar 6, 2020",2020-03-06 00:00:00.000000,SBI has already obtained an 'in-principle' app...,https://theprint.in/economy/state-bank-of-indi...,"data:image/gif;base64,R0lGODlhAQABAIAAAP//////...","[input_ids, token_type_ids, attention_mask]","{'positive': 0.07688067108392715, 'negative': ...",neutral
3,Fitch Affirms State Bank of India's IDR at 'BB...,NaN,"Apr 30, 2020",2020-04-30 00:00:00.000000,Fitch Ratings - Singapore - 30 Apr 2020: Fitch...,https://www.fitchratings.com/research/banks/fi...,"data:image/gif;base64,R0lGODlhAQABAIAAAP//////...","[input_ids, token_type_ids, attention_mask]","{'positive': 0.48873522877693176, 'negative': ...",neutral
4,SBI cautions against fake UPI ID for collectin...,Livemint,"Mar 31, 2020",2020-03-31 00:00:00.000000,Country's largest lender State Bank of India (...,https://www.livemint.com/industry/banking/sbi-...,"data:image/gif;base64,R0lGODlhAQABAIAAAP//////...","[input_ids, token_type_ids, attention_mask]","{'positive': 0.03070661798119545, 'negative': ...",neutral


In [14]:
SBI_news_mod['sentiment_news'].value_counts()

neutral     145
positive     49
negative     46
Name: sentiment_news, dtype: int64